In [94]:
import pandas as pd
import os
from sklearn.impute import KNNImputer
import numpy as np

In [13]:
base_dir = os.path.join(os.environ['HOME'], 'data', 'HCP-disease', 'behavioural')

In [29]:
banda_df = pd.read_excel(os.path.join(base_dir, 'BANDA', 'BehaviouralFiles.xlsx'), sheet_name=None)

In [ ]:
sub_scores = {
    "social_phobia": [4, 7, 8, 12, 20, 30, 32, 38, 43],
    "panic_disorder": [3, 14, 24, 26, 28, 34, 36, 39, 41],
    "maj_depression": [2, 6, 11, 15, 19, 21, 25, 29, 40, 47],
    "seperation_anxiety": [5, 9, 17, 18, 33, 45, 46],
    "general_anxiety": [1, 13, 22, 27, 35, 37],
    "ocd": [10, 16, 23, 31, 42, 44]
}

rdacs_score = {
    "Never": 0,
    "Sometimes": 1,
    "Often": 2,
    "Always": 3
}
rcads_cols = banda_df["rcads01"].columns[8:]
scored_df = banda_df["rcads01"][rcads_cols].replace(rdacs_score)
scores = dict(zip([key for key in sub_scores], [[] for key in sub_scores]))
scores['particpant'] = banda_df["rcads01"]['src_subject_id'].loc[1:]

for sub_scal in sub_scores:
    scores[sub_scal] = scored_df.loc[1:, [f"rcads_{str(index)}" for index in sub_scores[sub_scal]]].sum(axis=1)

rcads_df = pd.DataFrame(
    data = scores
)
cols = rcads_df.columns.tolist()
cols = [cols[-1]] + cols[1:-1] + [cols[0]]
cols = cols[:1] + [cols[2]] + [cols[1]] + cols[3:]
rcads_df = rcads_df[cols]

try:
    del(rcads_df['internalising'])
    del(rcads_df['anxiety'])
except Exception as e:
    pass
rcads_df['internalising'] = rcads_df.iloc[:, 1:].sum(axis=1)
rcads_df['anxiety'] = rcads_df.iloc[:, 2:7].sum(axis=1)

In [99]:
mfq_cols = banda_df['mfq01'].columns[8:41]
mfg_score = banda_df['mfq01'][mfq_cols].iloc[1:, :]
mfg_score

,mfqc1_1,mfqc1_2,mfqc1_3,mfqc1_4,mfqc1_5,mfqc1_6,mfqc2_1,mfqc2_2,mfqc2_3,mfqc2_4,...,mfqc4_6,mfqc5_1,mfqc5_2,mfqc5_3,mfqc5_4,mfqc5_5,mfqc6_1,mfqc6_2,mfqc6_3,mfqc6_4
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,2,1,2,1
2,1,1,1,1,2,1,1,1,1,1,...,1,2,1,1,1,1,1,1,1,2
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,2,1,1,1,1,2,1
5,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288,1,1,1,2,3,2,1,1,2,1,...,1,1,1,1,1,1,1,1,1,1
1289,1,2,2,1,2,1,1,1,1,1,...,1,1,2,1,1,1,1,1,1,1
1290,2,1,2,2,2,2,1,1,2,1,...,1,2,1,2,2,1,2,1,2,2
1291,2,2,1,2,2,1,2,2,1,3,...,1,1,1,3,1,2,1,2,2,3


In [ ]:

transformed = KNNImputer(
        missing_values=np.nan, 
                             weights="uniform"
                             ).fit_transform(mfg_score.shape[1])
    mfg_score = transformed

ValueError: Expected 2D array, got 1D array instead:
array=[1. 1. 1. ... 2. 2. 2.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.